# Journal de bord

Copier-coller les cellules comme canevas pour y remplir les informations.

## 📅 Journal de bord – Jour 1 (17/12/2025)

### 🔧 Étapes / Actions réalisées
- Découverte du sujet,
- Récupération des ressources,
- Structuration des dossiers
- Mise en place du tracking **git**

### 📝 Observations / Difficultés
- 

### ✅ Prochaines étapes
- 